In [2]:
import json
import os
import sys
import requests

In [3]:
with open("keys.json") as f:
    keys = json.load(f)

open_ai_key = keys["OPEN_AI_KEY"]
n8n_key = keys["N8N_KEY"]

In [4]:
from openai import OpenAI

def get_response(messages):
    client = OpenAI(api_key=open_ai_key)
    completion = client.chat.completions.create(model='gpt-4o-mini', messages=messages)
    return completion.choices[0].message.content


test_messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": "What is the capital of France?"
    }
]
get_response(test_messages)

'The capital of France is Paris.'

In [5]:

def create_workflow(name, nodes = [], connections = {}):
    url = "https://n8n.squarelight.ai/api/v1"

    workflow = {
        "name": name,
        "nodes": nodes,
        "connections": connections,
        "settings": {
            "saveExecutionProgress": True,
            "saveManualExecutions": True,
            "saveDataErrorExecution": "all",
            "saveDataSuccessExecution": "all",
            "executionTimeout": 3600,
            "errorWorkflow": "VzqKEW0ShTXA5vPj",
            "timezone": "America/New_York",
            "executionOrder": "v1"
        },
    }

    response = requests.post(f"{url}/workflows", headers={"X-N8N-API-KEY": n8n_key}, json=workflow)
    if response.status_code != 200:
        print(response.text)
        return None
    return response.json()

    


In [6]:
work_flow = create_workflow("Mina is so cool")
print(work_flow['id'])

T1QXLClNDwOhIe9l


In [7]:
work_flow['id']

'b4kBCGcOZbLKbUcm'

In [8]:
NODE_TYPES = {
    "n8n-nodes-base":[
        "n8n-nodes-base.manualTrigger"
    ],
    "@n8n/n8n-nodes-langchain":[
        "@n8n/n8n-nodes-langchain.agent",
        "@n8n/n8n-nodes-langchain.lmChatOpenAi"
    ]
}

def create_node(name, position, type, parameters = {}):
    """
    Create a node object

    Args:
    name (str): The name of the node
    position (list): The position of the node on the canvas
    type (str): The type of the node
    """

    return {
        "name": name,
        "type": type,
        "position": position,
        "parameters": parameters
    }

In [9]:
test_nodes = [
    create_node("Start", [0,0], "n8n-nodes-base.manualTrigger"),
    create_node("OpenAI", [200,0], "@n8n/n8n-nodes-langchain.agent"),
    create_node("OpenAI2", [200,0], "@n8n/n8n-nodes-langchain.agent"),
]


test_connections = {
    "Start": {
      "main": [
        [          {
            "node": "OpenAI",
            "type": "main",
            "index": 0
          },
                    {
            "node": "OpenAI2",
            "type": "main",
            "index": 0
          }

        ]
      ]
    }
  }

work_flow = create_workflow("Test with adding a node", test_nodes, test_connections)

In [17]:
system_prompt = """ You are a model created to generate workflow on n8n using their API.
This is the following functions and code you have access to

```python
def create_workflow(name, nodes = [], connections = {})
```

This function will create a new workflow with the given name, nodes and connections

```python
create_node(name, position, type, parameters = {})
```

This function will return a new node with the given name, position (list), type (string) and parameters (dict)

Here is an incomplete list of valid node types:
```python
NODE_TYPES = {
    "n8n-nodes-base":[
        "n8n-nodes-base.manualTrigger"
    ],
    "@n8n/n8n-nodes-langchain":[
        "@n8n/n8n-nodes-langchain.agent",
        "@n8n/n8n-nodes-langchain.lmChatOpenAi"
    ]
}
```

This is just an example of what node types look like and not meant to be exhaustive.

The connections object is an ordered map.

Here is a simple example of a manual trigger node connected to an OpenAI node

```python
test_nodes = [
    create_node("Start", [0,0], "n8n-nodes-base.manualTrigger"),
    create_node("OpenAI", [200,0], "@n8n/n8n-nodes-langchain.agent"),
]


test_connections = {
    "Start": {
      "main": [
        [
          {
            "node": "OpenAI",
            "type": "main",
            "index": 0
          }
        ]
      ]
    }
  }

work_flow = create_workflow("Test with adding a node", test_nodes, test_connections)
```
"""


In [18]:
planning_system = """
Main task now:
Create a detailed plan for how you would make a workflow that does the following:
<task>
Important things to consider:
- triggers: what will start the workflow?
- nodes: what nodes will you need?
- connections: how will the nodes be connected?"""

excuting_system = """" The following plan was formulated:
<plan>
Using this plan, return the respective code that would create the workflow. Respond with code only"""

In [20]:
task = "Create an example workflow that checks if someone commented on a post and then replies to them"

messages = [
    {
        "role": "system",
        "content": system_prompt+planning_system.replace("<task>", task)
    }
]

print("planning...")
plan = get_response(messages)

messages = [
    {
        "role": "system",
        "content": system_prompt+excuting_system.replace("<plan>", plan)
    }
]
print("Creating workflow...")
response = get_response(messages)
print(response)

planning...
Creating workflow...
```python
# Create nodes
nodes = [
    create_node("Comment Trigger", [0, 0], "n8n-nodes-base.manualTrigger"),
    create_node("Check Comment", [200, 0], "@n8n/n8n-nodes-langchain.commentCheckNode"),  # Assuming a check node exists
    create_node("Send Reply", [400, 0], "@n8n/n8n-nodes-langchain.sendReplyNode"),  # Assuming a reply node exists
]

# Define connections
connections = {
    "Comment Trigger": {
        "main": [
            [
                {
                    "node": "Check Comment",
                    "type": "main",
                    "index": 0
                }
            ]
        ]
    },
    "Check Comment": {
        "main": [
            [
                {
                    "node": "Send Reply",
                    "type": "main",
                    "index": 0
                }
            ]
        ]
    }
}

# Create the workflow
workflow = create_workflow("Comment Reply Workflow", nodes, connections)
```


In [25]:
response = response.replace("```python", "")
response = response.replace("```", "")
exec(response)